## Introduction 

In this notebook we are going to open a root file and take a look at some of the data that is stored. This is a file with real LHC data. 

In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import uproot
import h5py
import yaml
import pickle 
import mplhep as hep
import awkward as ak
import sklearn.metrics as metrics

plt.rcParams['figure.dpi'] = 100

/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-pac

In [4]:
filename = "/uscms/home/jpearkes/eos/forAda/scouting_nano.root" # originally from root://hip-cms-se.csc.fi//store/user/pinkaew/scouting_nano_prod_golden/ScoutingPFRun3/crab_ScoutingPFRun3_Run2024D-v1_380385-380470_Golden/240517_205837/0000/scouting_nano_399.root

file = uproot.open(filename)

In [3]:
# Take a look at the file contents 
file.keys()

['Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']

In [4]:
# The important stuff is stored in "Events"
file["Events"]

<TTree 'Events' (1510 branches) at 0x7f40a5b3ff70>

In [10]:
file["Events"].keys()

['run',
 'luminosityBlock',
 'event',
 'bunchCrossing',
 'orbitNumber',
 'nScoutingJet',
 'ScoutingJet_nConstituents',
 'ScoutingJet_nCh',
 'ScoutingJet_nElectrons',
 'ScoutingJet_nMuons',
 'ScoutingJet_nNh',
 'ScoutingJet_nPhotons',
 'ScoutingJet_area',
 'ScoutingJet_chEmEF',
 'ScoutingJet_chHEF',
 'ScoutingJet_eta',
 'ScoutingJet_mass',
 'ScoutingJet_muEF',
 'ScoutingJet_neEmEF',
 'ScoutingJet_neHEF',
 'ScoutingJet_phi',
 'ScoutingJet_pt',
 'ScoutingJet_particleNet_prob_b',
 'ScoutingJet_particleNet_prob_bb',
 'ScoutingJet_particleNet_prob_c',
 'ScoutingJet_particleNet_prob_cc',
 'ScoutingJet_particleNet_prob_g',
 'ScoutingJet_particleNet_prob_undef',
 'ScoutingJet_particlenet_prob_uds',
 'nScoutingFatJet',
 'ScoutingFatJet_nConstituents',
 'ScoutingFatJet_nCh',
 'ScoutingFatJet_nElectrons',
 'ScoutingFatJet_nMuons',
 'ScoutingFatJet_nNh',
 'ScoutingFatJet_nPhotons',
 'ScoutingFatJet_area',
 'ScoutingFatJet_chEmEF',
 'ScoutingFatJet_chHEF',
 'ScoutingFatJet_eta',
 'ScoutingFatJet_mas

In [9]:
ak.min(file["Events"]["ScoutingJet_mass"].array()) #Why is this negative?

-0.7373047

Different fields correspond to different types of data in the event.

Here the main ones we will be using are: 
- Scouting jets (ScoutingJet)
- Scouting electrons (ScoutingElectron)
- Scouting muons no (ScoutingMuonNoVtx)
- Level 1 Jets (L1Jet)
- Level 1 Electron/Photons (L1EG)
- Level 1 Muon (L1Muon)

The "L1" prefix means that these are objects that are reconstructed in electronics at the Level 1 Trigger. 
The "Scouting" prefix means that these are objects that are reconstructed in software at the High Level Trigger. (https://cms.cern/index.php/detector/triggering-and-data-acquisition)

## Questions to discuss
**What are jets, electrons, photons and muons?**
* Jets: Cone of hadrons (composite particles, baryons or mesons - odd quark # vs even) from "hadronization of quarks and gluons" -> hadronization happens because quarks and gluons have a color charge, and QCD confinement only allows for colorless states. Color charges determine what strong force interaction particles will have with one another. Basically, jets are just big spews of hadrons. Seen in hadron calorimeter 
* Electrons: Electrically charged fundamental particles (leptons), seen in EM calorimeter, subject to electric and magnetic fields
* Photons: Neutral, massless "particles" that regulate the electromagnetic force, seen in EM calorimeter
* Muons: Negatively charged leptons, more massive than electrons, come from collisions between particles + cosmic rays (or proton-tungsten collisions in the [Mu2e](https://mu2e.fnal.gov/how_does_it_work.shtml) ? pions -> muons), detected by the muon chambers

**What parts of the CMS detector are used for identifying these objects** https://cms.cern/detector/
* Jets: Hadron calorimeter (charged hadrons bend, neutral ones do not)
* Electrons + Photons: EM calorimeter
* Muons: Muon chambers

Visit [CMS Detector Slice](https://cds.cern.ch/record/2120661) for visual aid!

---
The most important variables we will be looking at are the pT, eta, phi and mass of each of these objects. For example 'ScoutingJet_pt',''ScoutingJet_eta','ScoutingJet_phi','ScoutingJet_mass'. 

See https://tikz.net/axis3d_cms/ for an illustration of the CMS co-ordinate system. 

## Questions to discuss
**What is pT and why is it useful?**

pT is the momentum component that is perpendicular/transverse to the beam line (the z axis). Using just pT guarantees that you're only paying attention to the physics at the collision point (vertex?) as opposed to the momentum along the beamline, which could be from other beam particles.
Source: ["Study of Transversal Momentum, Phi, and Eta"](https://www.i2u2.org/elab/cms/posters/display.jsp?type=paper&name=study_of_transversal_momentum_phi_and_eta-cms-daisy-fetsko-mills_godwin_high_school-henrico-va-2014.0302.data)

**What is the value of eta if a particle passes close to the beam-pipe? What is eta if the particle is perpendicular to the beampipe?** (https://en.wikipedia.org/wiki/Pseudorapidity)

Eta is the "pseudorapidity," which defines the angle of the particle in relation to the beam axis. The lower eta is, the closer to the beam it is, making it harder to distinguish. If eta is truly perpendicular to the beamline, it is infinite, because of its relationship with theta -> eta = ln(tan(theta/2)). Theta is the angle between the beamline and momentum. Sources: ["CMS coordinate system"](https://tikz.net/axis3d_cms/) and ["Pseudorapidity"](https://en.wikipedia.org/wiki/Pseudorapidity)

**What is phi and mass?**

Phi is the angle between the x axis (pointing towards the center of the LHC) and the pT vector. It is also known as the azimuthal (azimuth = "angle formed between a reference direction and a line from the observer to a point of interest". Source:["Azimuth"](https://en.wikipedia.org/wiki/Azimuth#:~:text=The%20azimuth%20is%20the%20angle%20formed%20between%20a,as%20the%20reference%20direction%20orthogonal%20to%20the%20zenith.) ) and/or scattering angle. Graphs of phi vs. eta can represent the direction of an outgoing particle and help determine the distance between two particles. 
Source: ["Pseudo-Rapidity, Azimuthal Angle, and Transverse Momentum"](https://www.phys.ufl.edu/~rfield/cdf/chgjet/etaphi.html)

Particle mass is measured in electronvolts, rather than an SI unit like kg, because they are so incredibly small. Known equations, like E^2 = (mc^2)^2 + (pc)^2, relate energy, momentum, and mass, allowing us to determine the mass of particles from energy signatures in the detector. We can also use the "kinematic characteristics" of measured particles to reconstruct the mass of their parent particle.
Sources: ["How CMS Detect Particles"](https://indico.cern.ch/event/464777/contributions/1971633/attachments/1197560/1742904/cmsDetector_detectParticles.pdf) and ["Measuring Particle Masses"](https://van.physics.illinois.edu/ask/listing/1209#:~:text=Typically%2C%20subatomic%20particle%20masses%20are%20determined%20by%20the,momentum%20and%20c%20is%20the%20speed%20of%20light.)

---

We can inspect these object like this: 

In [6]:
# LHC run number
jet_pt = file["Events"]["ScoutingJet_pt"].array() # Takes file tree, looks at events, looks at data within events, presents it in array
jet_pt

<Array [[], [26.7], ..., [23.4, 23.4, 21.3]] type='431555 * var * float32'>

Each row here corresponds to an "event", also known as a collision. During a collision different numbers of jets can be produced. Here, the first collision produced 0 jets, the second collision produced one jet with pt=26.7 GeV, and the 6th collision produced 5 jets!

We can use the awkward array and numpy libraries to perform some calculations on the jet pt. (References: https://awkward-array.org/doc/main/reference/generated/ak.mean.html & https://numpy.org/doc/stable/reference/generated/numpy.median.html) 

In [7]:
ak.num(jet_pt, axis=1) # total number of jets per event

<Array [0, 1, 1, 2, 0, 5, 4, 3, ..., 4, 3, 3, 6, 1, 5, 3] type='431555 * int64'>

In [8]:
ak.sum(jet_pt, axis=1) # total pt of jets per event

<Array [0, 26.7, 24.9, 181, 0, ..., 144, 26, 433, 68.1] type='431555 * float32'>

In [9]:
ak.mean(jet_pt) # mean jet pt per event

53.67741283067713

In [10]:
np.median(ak.flatten(jet_pt)) # median jet pt per event 

29.899792

# Questions to answer: 

**How many collision events are in this file?**

431,555 events 

**What is the maximum jet pt in an event** (https://awkward-array.org/doc/main/reference/generated/ak.max.html)

The maximum jet pt in any event is 1235.5298 GeV.

**What is the minimum jet pt in an event?**

The minimum jet pt in any event is 20.0 GeV.

**What are the mean, median, maximum and minimum values of jet pt, eta, phi and mass?**

Jet pt:

* Mean: 53.67741283067713

* Median: 29.899791717529297

* Maximum: 1235.52978515625

* Minimum: 20.0

Eta:

* Mean: 0.010506778174641298

* Median: 0.025064468383789062

* Maximum: 2.95703125

* Minimum: -2.978515625
 
Phi:

* Mean: 0.005393391787886657

* Median: 0.002682924270629883

* Maximum: 3.1416015625

* Minimum: -3.1416015625

Mass:

* Mean: 8.601295693829671

* Median: 6.55078125

* Maximum: 149.75

* Minimum: -0.7373046875

**What are the ranges of ScoutingElectron pt, eta, phi and mass, and ScoutingMuonNoVtx pt, eta, phi and mass? You may want to write a for loop to extract all of these.** 

Scouting Electron:

* pt Range: (3.9863281, 745.5)

* eta Range: (-2.5, 2.5)

* phi Range: (-3.140625, 3.140625)

* m Range: (-2.95043e-05, 2.3931265e-05)

ScoutingMuonNoVtx:

* pt Range: (1.0, 74688.0)

* eta Range: (-2.4003906, 2.4003906)

* phi Range: (-3.140625, 3.140625)

* m Range: (-0.0003452301, 0.0003452301)

See https://uproot.readthedocs.io/en/latest/basic.html for more information on opening and inspecting root files. 

In [29]:
# How many collision events? I think 431555, but let's find out!
tot = ak.num(jet_pt,axis = 0) # Every event has a jet_pt value, even if it is zero. axis = 0 counts number of events, not considering number of entries in each row.
tot

431555

In [66]:
# What is the max jet pt in an event?
tot_max = ak.max(jet_pt) # don't add an axis because we want the max out of all events
tot_max

1235.5298

In [37]:
# What is each event's max jet pt?
each_max = ak.max(jet_pt, axis = 1)
each_max

<Array [None, 26.7, 24.9, 97.6, ..., 26, 211, 23.4] type='431555 * ?float32'>

In [39]:
# What is the overall minimum jet pt?
tot_min = ak.min(jet_pt)
tot_min

20.0

In [40]:
# What is the minimum of each event?
each_min = ak.min(jet_pt, axis = 1)
each_min

<Array [None, 26.7, 24.9, 83.6, ..., 26, 24.8, 21.3] type='431555 * ?float32'>

In [8]:
# What are the mean, median, maximum and minimum values of jet pt, eta, phi and mass?
scouting_jet = {
    "jet pt" : file["Events"]["ScoutingJet_pt"].array(),
    "eta" : file["Events"]["ScoutingJet_eta"].array(),
    "phi" : file["Events"]["ScoutingJet_phi"].array(),
    "mass" : file["Events"]["ScoutingJet_mass"].array()
}

for key, value in scouting_jet.items():
    print(key)
    print(f"Mean: {ak.mean(value)}")
    print(f"Median: {np.median(ak.flatten(value))}")
    print(f"Maximum: {ak.max(value)}")
    print(f"Minimum: {ak.min(value)}")
    print("    ")

# I'm sure there's a more efficient way to do this

jet pt
Mean: 53.67741283067713
Median: 29.899791717529297
Maximum: 1235.52978515625
Minimum: 20.0
    
eta
Mean: 0.010506778174641298
Median: 0.025064468383789062
Maximum: 2.95703125
Minimum: -2.978515625
    
phi
Mean: 0.005393391787886657
Median: 0.002682924270629883
Maximum: 3.1416015625
Minimum: -3.1416015625
    
mass
Mean: 8.601295693829671
Median: 6.55078125
Maximum: 149.75
Minimum: -0.7373046875
    


In [96]:
# What are the ranges of ScoutingElectron pt, eta, phi and mass, and ScoutingMuonNoVtx pt, eta, phi and mass?
# You may want to write a for loop to extract all of these.
def minmax(arr):
    min = ak.min(arr)
    max = ak.max(arr)
    return (min, max)

scouting_em = {
    "electron pt" : file["Events"]["ScoutingElectron_pt"].array(),
    "electron eta" : file["Events"]["ScoutingElectron_eta"].array(),
    "electron phi" : file["Events"]["ScoutingElectron_phi"].array(),
    "electron m" : file["Events"]["ScoutingElectron_m"].array(), # No ScoutingElectron_mass?
    "muon pt" : file["Events"]["ScoutingMuonNoVtx_pt"].array(),
    "muon eta" : file["Events"]["ScoutingMuonNoVtx_eta"].array(),
    "muon phi" : file["Events"]["ScoutingMuonNoVtx_phi"].array(),
    "muon m" : file["Events"]["ScoutingMuonNoVtx_m"].array() # Same thing here?
}

for key, value in scouting_em.items(): 
    print(f"{key} range: {minmax(value)}")

electron pt range: (3.9863281, 745.5)
electron eta range: (-2.5, 2.5)
electron phi range: (-3.140625, 3.140625)
electron m range: (-2.95043e-05, 2.3931265e-05)
muon pt range: (1.0, 74688.0)
muon eta range: (-2.4003906, 2.4003906)
muon phi range: (-3.140625, 3.140625)
muon m range: (-0.0003452301, 0.0003452301)
